In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import tiktoken
from datasets import load_dataset
from tqdm import tqdm
import dataset_creators.config as config
import pandas as pd
import os
import uuid
import tqdm

In [3]:
dataset = load_dataset("ccdv/pubmed-summarization", split="test", cache_dir = '/scratch/ramprasad.sa/huggingface_datasets')
    
    
    

No config specified, defaulting to: pubmed-summarization/section
Found cached dataset pubmed-summarization (/scratch/ramprasad.sa/huggingface_datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b)


"anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ] . \n although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ] . yet , \n our current understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains meager and lags far behind that of depression . \n overall , neuropsychiatric symptoms in pd have been shown to be negatively associated with cognitive performance . \n for example , higher depression scores have been correlated with lower scores on the mini - mental state exam ( mmse ) [ 8 , 9 ] as well as tests of memory and executive functions ( e.g. , attention ) [ 1014 ] . \n likewise , apathy and anhedonia in pd patients have been associated with executive d

In [26]:
from datasets import load_dataset

dataset = load_dataset("Blaise-g/SumPubmed", split = "test", cache_dir = '/scratch/ramprasad.sa/huggingface_datasets')

# dataset = load_dataset("ccdv/pubmed-summarization", split="test", cache_dir = '/scratch/ramprasad.sa/huggingface_datasets')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/26147 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/3268 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /scratch/ramprasad.sa/huggingface_datasets/Blaise-g___parquet/Blaise-g--SumPubmed-24235d54ca935614/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [27]:
dataset

Dataset({
    features: ['line_text', 'filename_text', 'text', 'shorter_abstract', 'abstract'],
    num_rows: 3269
})

In [29]:
print(dataset['text'][0])

 evolutionary molecular biology is mostly concerned with the forces affecting individual genes. however, observations of variable proportions of guanine and cytosine in different species and in different genomic regions of vertebrates have prompted the analysis of forces that may affect the evolution of complete genomes. one particular hypothesis concerns adaptation to high temperatures, proposing that high gc content results from selection favouring g:c pairs over less stable a:t pairs. against initial expectations, there seems to be no direct relationship between the gc content of prokaryotic protein-coding genes and optimal growth temperature. similarly, in the case of vertebrates, it was argued convincingly that the'isochore' structure of high- and low-gc regions is not due to selection, but reflects varying fixation biases of gc over at pairs in the presence of recombination. a clear picture of selection at work emerges only in the study of structured rnas. the ribosomal rnas and 

In [10]:

def check_prompt_token_limits(article, instructions, tokenizer, token_limit):
    counter = 0
    for key, instruction in instructions.items():
        prompt = f'Article: {article}\n{instruction}'
        prompt_len = len(tokenizer.encode(prompt))
        if prompt_len < token_limit:
            counter += 1 
    return counter 

def get_shortlisted_data(articles, reference_summaries, ids = [], token_limit = 4096, dataset = 'xsum'):
    
    flan_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
    flan_instructions = config.instructions[f'{dataset}_flant5']
    
    gpt_tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    gpt_instructions = config.instructions[f'{dataset}_gpt3']
    
    shortlisted_articles = []
    shortlisted_reference_summaries = []
    
    shortlisted_ids = []
    
    for idx, article in enumerate(tqdm(articles)):
        add_article = 0 
        
        flan_counter = check_prompt_token_limits(article, flan_instructions, flan_tokenizer, token_limit)
        add_article += flan_counter
        
        gpt_counter = check_prompt_token_limits(article, gpt_instructions, gpt_tokenizer, token_limit)
        add_article += gpt_counter
        

        if add_article == 4:
            shortlisted_articles.append(article)
            shortlisted_reference_summaries.append(reference_summaries[idx])
            
            if not ids:
                article_id = str(uuid.uuid4())
            else:
                article_id = ids[idx]
            
            shortlisted_ids.append(article_id)
            
    return shortlisted_articles, shortlisted_reference_summaries, shortlisted_ids


def make_sample_pubmed(data_path, token_limit = 4096):
    dataset = load_dataset("ccdv/pubmed-summarization", split="test", cache_dir = '/scratch/ramprasad.sa/huggingface_datasets')
    articles = dataset['article']
    reference_summaries = dataset['abstract']
    
    
    
    shortlisted_data = {'article': [], 'reference_summary': [], 'id': [], 'origin': []}
    
    shortlisted_articles, shortlisted_reference_summaries, shortlisted_ids = get_shortlisted_data(articles, reference_summaries, dataset = 'pubmed')
    shortlisted_data['article'] += shortlisted_articles
    shortlisted_data['reference_summary'] += shortlisted_reference_summaries
    shortlisted_data['id'] += shortlisted_ids
    shortlisted_data['origin'] += ['pubmed'] * len(shortlisted_ids)
            
    isExist = os.path.exists(data_path)
    if not isExist:
        os.makedirs(data_path)
    
    df = pd.DataFrame(shortlisted_data)
    df.to_csv(f'{data_path}/test_sample.csv')
    return df



In [11]:
shortlisted_data = make_sample_pubmed('/home/ramprasad.sa/factual_annotation_llm_summaries/datasets/pubmed')
# pd.DataFrame(shortlisted_data)

No config specified, defaulting to: pubmed-summarization/section
Found cached dataset pubmed-summarization (/scratch/ramprasad.sa/huggingface_datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b)
  1%|█▊                                                                                                                                                                                                                                       | 51/6658 [00:01<04:05, 26.97it/s]

KeyboardInterrupt



In [8]:
shortlisted_data

,article,reference_summary,id,origin
0,congenital adrenal hyperplasia ( cah ) refers ...,congenital adrenal hyperplasia is a group of a...,f05ee546-a3c4-40fb-b604-9497cb165de8,pubmed
1,the family is the cornerstone of human social ...,background : since the family is a social syst...,7bca5d77-7541-4ff9-a46f-d446b72dd9cd,pubmed
2,development of human societies and industriali...,background and objective : anxiety and depre...,0fc68a72-a6d3-4ec9-8f43-61124446a064,pubmed
3,male macroprolactinomas ( mprl ) are usually r...,background : suppurative meningitis ( sm ) or ...,86ed3313-6fde-46d4-b555-0afd6821f378,pubmed
4,the femoral head often leads to healing compli...,fracture of the femoral neck continues to be a...,861ad9de-1ed4-4dc2-94f3-b4ec29b8e99e,pubmed
...,...,...,...,...
3499,odorant binding proteins ( obps ) were identif...,odorant - binding proteins ( obps ) were disco...,543ed531-f820-48f4-ba82-bb7cd18b0f56,pubmed
3500,minimally invasive surgery was first described...,purposethis study aimed to comparatively evalu...,29d98812-1c9e-456b-9e7c-1836a5d7c478,pubmed
3501,a 60-year - old male patient was referred to a...,warthin 's tumor is the second most common typ...,e7456860-d6cc-4811-955b-69eea48b860c,pubmed
3502,they almost always require open reduction and ...,introduction : subtrochanteric fractures have ...,bc18fe1d-dd83-4523-a52c-b9bf465b8d78,pubmed


In [6]:
len(dataset['abstract'])

6658

In [9]:
dataset['abstract'][0]

"research on the implications of anxiety in parkinson 's disease ( pd ) has been neglected despite its prevalence in nearly 50% of patients and its negative impact on quality of life . \n previous reports have noted that neuropsychiatric symptoms impair cognitive performance in pd patients ; however , to date , no study has directly compared pd patients with and without anxiety to examine the impact of anxiety on cognitive impairments in pd . \n this study compared cognitive performance across 50 pd participants with and without anxiety ( 17 pda+ ; 33 pda ) , who underwent neurological and neuropsychological assessment . \n group performance was compared across the following cognitive domains : simple attention / visuomotor processing speed , executive function ( e.g. , set - shifting ) , working memory , language , and memory / new verbal learning . \n results showed that pda+ performed significantly worse on the digit span forward and backward test and part b of the trail making task

In [2]:
df = pd.read_csv('/home/ramprasad.sa/factual_annotation_llm_summaries/datasets/pubmed/test_sample.csv')
df

,Unnamed: 0,article,reference_summary,id,origin
0,0,one way to organize the billions of cells wit...,stem cells divide to reproduce themselves and...,aaee690b-2ccf-467a-8f46-74899e2e1a28,pubmed
1,1,species identification is essential for large...,the rapid and accurate identification of spec...,9d8cd34c-00cc-44d6-8c18-ea5e4a354292,pubmed
2,2,rhodococcus equi is one of the most significa...,rhodococcus equi is one of the most significa...,897f6c53-bf0d-4396-8e48-0b8b5ea65252,pubmed
3,3,"the galliformes, a well-known and widely dist...",the galliformes is a well-known and widely di...,1a5a6780-5922-4e14-b76c-fe4726161fa5,pubmed
4,4,light drives the processes of photosynthesis ...,"although specific light attributes, such as c...",f5c29a93-4198-49bb-992d-4a384579ab53,pubmed
...,...,...,...,...,...
580,580,the increased use of high-throughput analysis...,in response to the frequently overwhelming ou...,87b4c2a9-c773-4bcf-bcb6-7fdbc3653b45,pubmed
581,581,evolution leads to genetic diversity the sele...,non-synonymous single nucleotide polymorphism...,683b71f8-9f02-4ab9-bbc0-9aa3312fa336,pubmed
582,582,although different experimental methods have ...,protein-protein interactions are crucial in c...,20803371-fed1-438a-992c-fa123673229f,pubmed
583,583,"scarless, site-directed mutagenesis on a bact...",a variety of techniques have been described w...,f8e7647b-eb91-4bf2-996e-57d046d4bb2d,pubmed


In [19]:
import pandas as pd

In [20]:
df = pd.read_csv('/home/ramprasad.sa/factual_annotation_llm_summaries/datasets/pubmed/test_sample.csv')


In [1]:
import accelerate

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import accelerate

# Prepare deepspeed
deepspeed = accelerate.DeepSpeedPlugin(gradient_accumulation_steps=1,
                                       zero3_init_flag=False,
                                       zero_stage=3,
                                       offload_optimizer_device="cuda",
                                       offload_param_device="cpu")
accelerator = accelerate.Accelerator(deepspeed_plugin=deepspeed)
accelerate.state.AcceleratorState().deepspeed_plugin.deepspeed_config['train_micro_batch_size_per_gpu'] = 1
accelerate.state.AcceleratorState()

ImportError: DeepSpeed is not installed => run `pip install deepspeed` or build it from source.

In [12]:
len(df['article'].values[0].split(' '))

2512

In [36]:
article = df['article'].values[0]
instruction = 'Summarize the above article'

In [37]:
from models.Flan_t5.flan_t5 import FlanT5Inference

In [38]:
model = FlanT5Inference()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [43]:
from accelerate import init_empty_weights
from transformers import AutoConfig,AutoModel 
checkpoint = "google/flan-t5-xxl"
config = AutoConfig.from_pretrained(checkpoint, cache_dir="/scratch/ramprasad.sa/huggingface_models")

with init_empty_weights():
    model = AutoModel.from_config(config)


NotImplementedError: Initializing empty weights to a meta device requires torch >= 1.9.0

In [1]:
 dataset = load_dataset("ccdv/pubmed-summarization", split="test", cache_dir = '/scratch/ramprasad.sa/huggingface_datasets')
    
    
    

NameError: name 'load_dataset' is not defined